In [74]:
from matplotlib import pyplot
from PIL import Image
import numpy as np
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = np.asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	# perform prediction
	yhat = model.predict(samples)
	np.save(open('data.stupi', 'wb'), yhat)
	# yhat = np.concatenate((train_data, yhat))
	
	return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))


In [73]:
filenames = ['images/prawit.jpg', 'images/prayut.jpg', 'images/anutin.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# verify known photos of sharon
# is_match(embeddings[0], embeddings[1])

>face is NOT a Match (0.638 > 0.500)


In [75]:
filenames = ['images/prawit.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# verify known photos of sharon
is_match(embeddings[0], embeddings[1])

>face is a Match (0.429 <= 0.500)


In [68]:
train_data = np.load(open('data.stupid', 'rb'))


In [ ]:
filenames = ['images/picture.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# verify known photos of sharon
is_match(embeddings[0], embeddings[1])

In [ ]:
import face_recognition
known_image = face_recognition.load_image_file("images/1.jpg")
unknown_image = face_recognition.load_image_file("images/2.jpg")

biden_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces([biden_encoding], unknown_encoding)
print(results)